#### Monte Carlo Simulation

In [24]:
# Initial imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np
import hvplot.pandas
import plotly.express as px


%matplotlib inline

In [25]:
# Load .env enviroment variables
load_dotenv()

True

In [26]:
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_SECRET_KEY")

In [27]:
# Set start and end dates of five years back from today.
# Formatting current date as ISO format
start_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-02-14", tz="America/New_York").isoformat()

In [28]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [29]:

# Setting the tickers
tickers = ["FB", "APPL","AMZN","NFLX","GOOG","SPY"]

# Setting timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Getting current closing prices for the tickers
df_tech_stock_data = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df


# Reorganize the DataFrame
# Separate ticker data
FB = df_tech_stock_data[df_tech_stock_data['symbol']=='FB'].drop('symbol', axis=1)
APPL = df_tech_stock_data[df_tech_stock_data['symbol']=='APPL'].drop('symbol', axis=1)
SPY = df_tech_stock_data[df_tech_stock_data['symbol']=='SPY'].drop('symbol', axis=1)
AMZN = df_tech_stock_data[df_tech_stock_data['symbol']=='AMZN'].drop('symbol', axis=1)
NFLX = df_tech_stock_data[df_tech_stock_data['symbol']=='NFLX'].drop('symbol', axis=1)
GOOG = df_tech_stock_data[df_tech_stock_data['symbol']=='GOOG'].drop('symbol', axis=1)


# Concatenate the ticker DataFrames
df_tech_stock_data = pd.concat([GOOG,APPL,SPY,AMZN,NFLX,FB],axis=1, keys=['GOOG','APPL','SPY','AMZN','NFLX','FB'])

# Preview DataFrame
df_tech_stock_data.head()
df_tech_stock_data.tail()



# df_etf = alpaca.get_bars(
#     tickers,
#     timeframe,
#     start = start_date,
#     end = end_date
# ).df

# #Tech
# FB = df_tech_stock_data[df_tech_stock_data['symbol']=='FB'].drop('symbol', axis=1)
# APPL = df_tech_stock_data[df_tech_stock_data['symbol']=='APPL'].drop('symbol', axis=1)
# SPY = df_tech_stock_data[df_tech_stock_data['symbol']=='SPY'].drop('symbol', axis=1)
# AMZN = df_tech_stock_data[df_tech_stock_data['symbol']=='AMZN'].drop('symbol', axis=1)
# NFLX = df_tech_stock_data[df_tech_stock_data['symbol']=='NFLX'].drop('symbol', axis=1)
# GOOG = df_tech_stock_data[df_tech_stock_data['symbol']=='GOOG'].drop('symbol', axis=1)

# #ETF
# # three_voo = df_etf[df_etf['symbol']=='VOO'].drop('symbol', axis=1)
# # three_vgt = df_etf[df_etf['symbol']=='VGT'].drop('symbol', axis=1)

# # Tech concatenate the ticker DataFrames
# df_stock_data = pd.concat([three_fb,three_appl,three_spy,three_amzn,three_fb,three_nflx,three_goog],axis=1, keys=['FB','APPL','AMZN','NFLX','GOOG','SPY'])
# # Tech concatenate the ticker DataFrames
# # df_etf_data = pd.concat([three_voo,three_vgt],axis=1, keys=['VOO','VGT'])


# # Preview DataFrame
# df_stock_data
# # df_etf_data

# # # Display sample data
# df_stock_data.head()
# # df_etf_data.head()

GOOG                                     \
                             open     high     low   close    volume   
timestamp                                                              
2023-02-08 05:00:00+00:00  102.69  103.580  98.455  100.00  73667738   
2023-02-09 05:00:00+00:00  100.54  100.610  93.860   95.46  97843496   
2023-02-10 05:00:00+00:00   95.74   97.020  94.530   94.86  49325275   
2023-02-13 05:00:00+00:00   95.01   95.350  94.050   95.00  43116559   
2023-02-14 05:00:00+00:00   94.66   95.175  92.650   94.95  42301435   

                                                  APPL           ...     NFLX  \
                          trade_count        vwap open high low  ...   volume   
timestamp                                                        ...            
2023-02-08 05:00:00+00:00      687581  100.292608  NaN  NaN NaN  ...  6257088   
2023-02-09 05:00:00+00:00      870413   95.558989  NaN  NaN NaN  ...  6904750   
2023-02-10 05:00:00+00:00      418697   95.366307  NaN  NaN NaN  ...  7291096   
2023-02-13 05:00:00+00:00      358711   94.762998  NaN  NaN NaN  ...  7148643   
2023-02-14 05:00:00+00:00      398556   94.088707  NaN  NaN NaN  ...  4557241   

                                                    FB                        \
                          trade_count        vwap open high low close volume   
timestamp                                                                      
2023-02-08 05:00:00+00:00      110837  364.534783  NaN  NaN NaN   NaN    NaN   
2023-02-09 05:00:00+00:00      118911  367.981924  NaN  NaN NaN   NaN    NaN   
2023-02-10 05:00:00+00:00      137480  352.290413  NaN  NaN NaN   NaN    NaN   
2023-02-13 05:00:00+00:00      121618  354.784016  NaN  NaN NaN   NaN    NaN   
2023-02-14 05:00:00+00:00       90300  358.780210  NaN  NaN NaN   NaN    NaN   

                                            
                          trade_count vwap  
timestamp                                   
2023-02-08 05:00:00+00:00         NaN  NaN  
2023-02-09 05:00:00+00:00         NaN  NaN  
2023-02-10 05:00:00+00:00         NaN  NaN  
2023-02-13 05:00:00+00:00         NaN  NaN  
2023-02-14 05:00:00+00:00         NaN  NaN  

[5 rows x 42 columns]

In [30]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
thirty_years_cumulative_returns = MCSimulation(
    portfolio_data = df_tech_stock_data ,
    weights=[.6,.4],
    num_simulation=500,
    num_trading_days=252*30
)
thirty_years_cumulative_returns

In [31]:
thirty_years_cumulative_returns.portfolio_data

GOOG                                          \
                              open      high        low    close    volume   
timestamp                                                                    
2020-01-02 05:00:00+00:00  1341.55  1368.140  1341.5500  1367.37   1406731   
2020-01-03 05:00:00+00:00  1347.86  1372.500  1345.5436  1360.66   1187006   
2020-01-06 05:00:00+00:00  1350.00  1396.500  1350.0000  1394.21   1733159   
2020-01-07 05:00:00+00:00  1397.94  1402.990  1390.3800  1393.34   1511693   
2020-01-08 05:00:00+00:00  1392.08  1411.580  1390.8400  1404.32   1529600   
...                            ...       ...        ...      ...       ...   
2023-02-08 05:00:00+00:00   102.69   103.580    98.4550   100.00  73667738   
2023-02-09 05:00:00+00:00   100.54   100.610    93.8600    95.46  97843496   
2023-02-10 05:00:00+00:00    95.74    97.020    94.5300    94.86  49325275   
2023-02-13 05:00:00+00:00    95.01    95.350    94.0500    95.00  43116559   
2023-02-14 05:00:00+00:00    94.66    95.175    92.6500    94.95  42301435   

                                                                APPL       \
                          trade_count         vwap daily_return open high   
timestamp                                                                   
2020-01-02 05:00:00+00:00       41393  1358.286088          NaN  NaN  NaN   
2020-01-03 05:00:00+00:00       28661  1363.616032    -0.004907  NaN  NaN   
2020-01-06 05:00:00+00:00       43017  1387.410598     0.024657  NaN  NaN   
2020-01-07 05:00:00+00:00       37866  1396.421509    -0.000624  NaN  NaN   
2020-01-08 05:00:00+00:00       38568  1403.643663     0.007880  NaN  NaN   
...                               ...          ...          ...  ...  ...   
2023-02-08 05:00:00+00:00      687581   100.292608    -0.074417  NaN  NaN   
2023-02-09 05:00:00+00:00      870413    95.558989    -0.045400  NaN  NaN   
2023-02-10 05:00:00+00:00      418697    95.366307    -0.006285  NaN  NaN   
2023-02-13 05:00:00+00:00      358711    94.762998     0.001476  NaN  NaN   
2023-02-14 05:00:00+00:00      398556    94.088707    -0.000526  NaN  NaN   

                           ...        NFLX                     FB          \
                           ...        vwap daily_return      open    high   
timestamp                  ...                                              
2020-01-02 05:00:00+00:00  ...  327.994995          NaN  206.7500  209.79   
2020-01-03 05:00:00+00:00  ...  327.435520    -0.011855  207.2135  210.40   
2020-01-06 05:00:00+00:00  ...  331.826407     0.030469  206.7000  212.78   
2020-01-07 05:00:00+00:00  ...  331.955986    -0.015127  212.8200  214.58   
2020-01-08 05:00:00+00:00  ...  337.815970     0.025729  213.0000  216.24   
...                        ...         ...          ...       ...     ...   
2023-02-08 05:00:00+00:00  ...  364.534783     0.010690       NaN     NaN   
2023-02-09 05:00:00+00:00  ...  367.981924    -0.011804       NaN     NaN   
2023-02-10 05:00:00+00:00  ...  352.290413    -0.041766       NaN     NaN   
2023-02-13 05:00:00+00:00  ...  354.784016     0.032272       NaN     NaN   
2023-02-14 05:00:00+00:00  ...  358.780210     0.003877       NaN     NaN   

                                                                     \
                                low   close      volume trade_count   
timestamp                                                             
2020-01-02 05:00:00+00:00  206.2700  209.78  12090707.0    101276.0   
2020-01-03 05:00:00+00:00  206.9500  208.67  11197416.0     98107.0   
2020-01-06 05:00:00+00:00  206.5200  212.60  17879436.0    118383.0   
2020-01-07 05:00:00+00:00  211.7500  213.06  15111857.0    114086.0   
2020-01-08 05:00:00+00:00  212.6121  215.22  13492703.0    107572.0   
...                             ...     ...         ...         ...   
2023-02-08 05:00:00+00:00       NaN     NaN         NaN         NaN   
2023-02-09 05:00:00+00:00       NaN     NaN         NaN         NaN  

In [32]:
thirty_years_cumulative_returns.calc_cumulative_return()

Running Monte Carlo simulation number 0.


ValueError: Dot product shape mismatch, (7561, 6) vs (2,)

In [ ]:
# Format current date as ISO format
start_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-02-14", tz="America/New_York").isoformat()
# Set the tickers
tickers = ["VOO", "VGT",'SPY']

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for VOO and VGT
df_etf = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Reorganize the DataFrame
# Separate ticker data
VOO = df_etf[df_etf['symbol']=='VOO'].drop('symbol', axis=1)
VGT = df_etf[df_etf['symbol']=='VGT'].drop('symbol', axis=1)
SPY = df_etf[df_etf['symbol']=='SPY'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
df_etf = pd.concat([VOO,VGT,SPY],axis=1, keys=['VOO','VGT','SPY'])

# Preview DataFrame
df_etf.head()
df_etf.tail()

VOO                                     \
                              open    high       low   close   volume   
timestamp                                                               
2023-02-08 05:00:00+00:00  379.690  380.93  376.7300  377.34  2922597   
2023-02-09 05:00:00+00:00  380.970  381.00  372.9486  374.11  3328606   
2023-02-10 05:00:00+00:00  373.000  375.39  372.2000  375.02  3256705   
2023-02-13 05:00:00+00:00  375.649  379.44  375.2000  379.35  2980600   
2023-02-14 05:00:00+00:00  377.960  381.42  375.4500  379.25  4526424   

                                                      VGT                    \
                          trade_count        vwap    open     high      low   
timestamp                                                                     
2023-02-08 05:00:00+00:00       67226  378.352946  369.48  372.209  364.370   
2023-02-09 05:00:00+00:00       68734  376.734588  370.70  371.360  361.700   
2023-02-10 05:00:00+00:00       63885  374.182561  360.35  362.015  357.690   
2023-02-13 05:00:00+00:00       63779  378.192463  362.78  368.100  362.075   
2023-02-14 05:00:00+00:00       68565  378.798354  365.44  371.210  363.130   

                           ...                                      SPY  \
                           ...   volume trade_count        vwap    open   
timestamp                  ...                                            
2023-02-08 05:00:00+00:00  ...  1275502       16721  366.700111  413.13   
2023-02-09 05:00:00+00:00  ...   390816        9381  366.758308  414.41   
2023-02-10 05:00:00+00:00  ...   337250        9901  360.105391  405.86   
2023-02-13 05:00:00+00:00  ...   978478       14683  366.446200  408.72   
2023-02-14 05:00:00+00:00  ...   499398       11286  368.484981  411.24   

                                                                          \
                             high      low   close    volume trade_count   
timestamp                                                                  
2023-02-08 05:00:00+00:00  414.53  409.930  410.65  76239095      529629   
2023-02-09 05:00:00+00:00  414.57  405.810  407.09  78709874      567083   
2023-02-10 05:00:00+00:00  408.44  405.010  408.04  70769715      489779   
2023-02-13 05:00:00+00:00  412.97  408.240  412.83  64913539      426599   
2023-02-14 05:00:00+00:00  415.05  408.511  412.64  80226538      617776   

                                       
                                 vwap  
timestamp                              
2023-02-08 05:00:00+00:00  411.925098  
2023-02-09 05:00:00+00:00  409.794927  
2023-02-10 05:00:00+00:00  407.008915  
2023-02-13 05:00:00+00:00  411.386908  
2023-02-14 05:00:00+00:00  412.071123  

[5 rows x 21 columns]

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
thirty_years_cumulative_returns = MCSimulation(
    portfolio_data = df_etf ,
    weights=[.6,.4],
    num_simulation=500,
    num_trading_days=252*30
)
thirty_years_cumulative_returns

In [ ]:
thirty_years_cumulative_returns.portfolio_data

VOO                                     \
                              open    high       low   close   volume   
timestamp                                                               
2020-01-02 05:00:00+00:00  297.290  298.45  296.3400  298.42  3149761   
2020-01-03 05:00:00+00:00  295.220  297.36  295.0400  296.24  3248060   
2020-01-06 05:00:00+00:00  294.420  297.42  294.3500  297.35  2782110   
2020-01-07 05:00:00+00:00  296.820  297.27  296.0611  296.53  2384732   
2020-01-08 05:00:00+00:00  296.710  299.31  296.4700  298.06  3724359   
...                            ...     ...       ...     ...      ...   
2023-02-08 05:00:00+00:00  379.690  380.93  376.7300  377.34  2922597   
2023-02-09 05:00:00+00:00  380.970  381.00  372.9486  374.11  3328606   
2023-02-10 05:00:00+00:00  373.000  375.39  372.2000  375.02  3256705   
2023-02-13 05:00:00+00:00  375.649  379.44  375.2000  379.35  2980600   
2023-02-14 05:00:00+00:00  377.960  381.42  375.4500  379.25  4526424   

                                                                   VGT  \
                          trade_count        vwap daily_return    open   
timestamp                                                                
2020-01-02 05:00:00+00:00       26846  297.278811          NaN  247.05   
2020-01-03 05:00:00+00:00       34024  296.477589    -0.007305  245.93   
2020-01-06 05:00:00+00:00       24415  295.912195     0.003747  244.43   
2020-01-07 05:00:00+00:00       20909  296.517868    -0.002758  247.87   
2020-01-08 05:00:00+00:00       24639  297.720142     0.005160  247.69   
...                               ...         ...          ...     ...   
2023-02-08 05:00:00+00:00       67226  378.352946    -0.010956  369.48   
2023-02-09 05:00:00+00:00       68734  376.734588    -0.008560  370.70   
2023-02-10 05:00:00+00:00       63885  374.182561     0.002432  360.35   
2023-02-13 05:00:00+00:00       63779  378.192463     0.011546  362.78   
2023-02-14 05:00:00+00:00       68565  378.798354    -0.000264  365.44   

                                    ...                              SPY  \
                              high  ...        vwap daily_return    open   
timestamp                           ...                                    
2020-01-02 05:00:00+00:00  249.360  ...  248.026042          NaN  323.54   
2020-01-03 05:00:00+00:00  248.390  ...  247.550554    -0.010508  321.16   
2020-01-06 05:00:00+00:00  247.630  ...  246.261497     0.003324  320.49   
2020-01-07 05:00:00+00:00  248.555  ...  247.898237    -0.000121  323.02   
2020-01-08 05:00:00+00:00  251.158  ...  249.664227     0.010545  322.94   
...                            ...  ...         ...          ...     ...   
2023-02-08 05:00:00+00:00  372.209  ...  366.700111    -0.012662  413.13   
2023-02-09 05:00:00+00:00  371.360  ...  366.758308    -0.004220  414.41   
2023-02-10 05:00:00+00:00  362.015  ...  360.105391    -0.006357  405.86   
2023-02-13 05:00:00+00:00  368.100  ...  366.446200     0.017005  408.72   
2023-02-14 05:00:00+00:00  371.210  ...  368.484981     0.005310  411.24   

                                                                          \
                             high      low   close    volume trade_count   
timestamp                                                                  
2020-01-02 05:00:00+00:00  324.89  322.530  324.87  59253833      304884   
2020-01-03 05:00:00+00:00  323.64  321.100  322.43  77783121      358500   
2020-01-06 05:00:00+00:00  323.73  320.360  323.73  55817168      255767   
2020-01-07 05:00:00+00:00  323.54  322.240  322.74  42874811      226059   
2020-01-08 05:00:00+00:00  325.78  322.670  324.42  68435753      340004   
...                           ...      ...     ...       ...         ...   
2023-02-08 05:00:00+00:00  414.53  409.930  410.65  76239095      529629   
2023-02-09 05:00:00+00:00  414.57  405.810  407.09  78709874      567083   
2023-02-10 05:00:00+00:00  408.44  405.010  408.04  70769715      4897